# Simulation parameters
We create a parameter dataframe containing the simulations' parameters, and a convenience function to fill it.

In [1]:
import math
import pandas as pd

# create empty dataframe
columns = ["name",  # name of the parameter
           "info",  # additional info, if required
           "real_value",  # real value (i.e. not using arbitrary units)
           "real_range_max",  # max theoretical value (if meaningful)
           "real_range_min",  # min theoretical value (if meaningful)
           "real_um",  # unit of measure
           "sim_value",  # value in arbitrary units
           "sim_range_max",  # max value in arbitrary units
           "sim_range_min",  # min value in arbitrary units
           "sim_um",  # arbitrary units of measure
           "references",  # scientific reference for the parameter (if needed)
           "influence"]  # list of other parameters affected by the current parameter (not used)
parameters_df = pd.DataFrame(columns=columns)


# define convenience function to fill parameters dict
def fill_parameters(params: pd.DataFrame,
                    name=None,
                    info=None,
                    real_value=None,
                    real_range_max=None,
                    real_range_min=None,
                    real_um=None,
                    sim_value=None,
                    sim_range_max=None,
                    sim_range_min=None,
                    sim_um=None,
                    references=None,
                    influence=None):
    # get input arguments
    args = locals()

    # define dictionary of the new row
    new_parameter_dict = {}
    for column in columns:
        new_parameter_dict[column] = [args[column]]

    # add it to
    return pd.concat([params, pd.DataFrame(new_parameter_dict)])

Then we initialize the arbitrary units:

In [2]:
from pint import UnitRegistry

# define unit registry
ureg = UnitRegistry()

# define names
sau_name = "SpaceArbitraryUnit"
tau_name = "TimeArbitraryUnit"
afau_name = "AFsArbitraryUnit"

# define arbitrary units for space and time
ureg.define(f"{sau_name} = 1 * mm = sau")
ureg.define(f"{tau_name} = 26 * min = tau")

# define arbitrary units
sim_s_au = 1 * ureg.mm
sim_t_au = 26 * ureg.min
sim_af_au = 1 * ureg("ng / mL")
ureg.define(f"{afau_name} = {sim_af_au} = afau")

# add arbitrary units to dataframe
parameters_df = fill_parameters(parameters_df,
                                name=sau_name,
                                real_value=sim_s_au.magnitude,
                                real_um=sim_s_au.units,
                                sim_value=sim_s_au.to("sau").magnitude,
                                sim_um=sim_s_au.to("sau").units)
parameters_df = fill_parameters(parameters_df,
                                name=tau_name,
                                real_value=sim_t_au.magnitude,
                                real_um=sim_t_au.units,
                                sim_value=sim_t_au.to("tau").magnitude,
                                sim_um=sim_t_au.to("tau").units)
parameters_df = fill_parameters(parameters_df,
                                name=afau_name,
                                real_value=sim_af_au.magnitude,
                                real_um=sim_af_au.units,
                                sim_value=1,
                                sim_um="afau")

In the following, we briefly discuss the derivation for each parameter.

> Note: some parameters' name might change from the manuscript to this Notebook

## Angiogenic factor (af) parameters

In our model we assume that the angiogenic factors (af) are generated by hypoxic cells distributed in the domain outside the capillaries. Following what proposed by Travasso and collaeagues, we assume that the af distribution is regulated by the following PDE:

$$
\begin{align}
\frac{\partial af}{\partial t} = D_{af}\nabla^2 af - V_{uC_{af}}\cdot af \cdot c \cdot shf(c)
\end{align}
$$

Where:
- $shf$ is a smooth Heavyside function
- $D_{af}$ is the AFs diffusivity
- $V_{uC_{af}}$ is the AFs uptake rate by the capillaries

So we presume that $af$ is diffusing in the domain, and is consumed by the capillaries. 

The production of af is due to the hypoxic cells (source cells), which are regulated by the following parameters:
- $R_c$, the radius of the hypoxic cells, assumed equal to the one of tip cells (see below).
- $n_source_cells0$, the initial number of the source cells.
- $source_cells_range$, the range where an hypoxic cell stops being hypoxic for the reach of oxygen.
- $af_min$, the value of the angiogenic factors outside the source cells.
- $af_max$, the value of the angiogenic factors inside the source cells.

We considered the vascular endothelial growth factor (VEGF) as the main reference to estimate AFs parameters.

### Angiogenic factor diffusivity ($D_{af}$)
We could not find a specific experimental value for VEGF diffusivity, but we estimated that from the experimental results of Kihara et al. (2013).

Kihara et al. (2013) measured the diffusivity of different peptides, with molecular weights ranging from 3 to 40 kDa. The diffusivity is measured in an actual extracellular matrix condensed by fibroblasts, making the experimental conditions closer to the retinal environment than a simple measure in water or serum.

In Table 1 they report the diffusion coefficients for two peptides with a mass close to VEGF:
- Alexa488 dextran (which is 10 kDa)
- and for FITC dextran (40 kDa)

In [3]:
D_Alexa488_dextran_serum = 81.7 * ureg("(um^2) / s")
D_FITC_dextran_serum = 44.7e-6 * ureg("(um^2) / s")

In Figure 8 they show that the diffusivity reduction in the extracellular matrix (ECM) compared to serum is quite small (27%), even though statistically significant.

In [4]:
D_Alexa488_dextran_matrix = 0.73 * D_Alexa488_dextran_serum
D_FITC_dextran_matrix = 0.73 * D_FITC_dextran_serum

We emplyed these measure to estimate VEGF diffusivity in retina by linear interpolation between the two values reported above.

According to the Uniprot entry for VEGFA (Uiprot ID: P15692) the VEGF isoforms have a molecular weight between 15,981 Da (Isoform VEGF111; identifier: P15692-10) and 45,467 Da (Isoform L-VEGF206; identifier: P15692-14). So we have:

In [5]:
# estimate as a line D = m * mol_w + q
m = (D_FITC_dextran_serum - D_Alexa488_dextran_serum) / (40e3 - 10e3)
q = D_Alexa488_dextran_serum - (10e3 * m)

# max diffusivity: smaller isoform in serum:
min_mol_w = 16e3  # Da
D_VEGF_max = (m * min_mol_w) + q
D_VEGF_max = D_VEGF_max.to("(mm^2) / s")  # convert to compare with others

# min diffusivity: bigger isoform in serum
max_mol_w = 45e3  # Da
D_VEGF_min = 0.73 * (m * max_mol_w) + q
D_VEGF_min = D_VEGF_min.to("(mm^2) / s")  # convert to compare with others

print(f"VEGF Diffusivity is in range: [{D_VEGF_min:.2g}, {D_VEGF_max:.2g}] millimiters^2 / seconds")

VEGF Diffusivity is in range: [1.9e-05 millimeter ** 2 / second, 6.5e-05 millimeter ** 2 / second] millimiters^2 / seconds


The range we found agrees with the values provided by the other authors.

In [6]:
D_af = (D_VEGF_min + D_VEGF_max) / 2
print("## D_af \n"
      f"value:  {D_af} \n")

# add it to dictionary
parameters_df = fill_parameters(parameters_df,
                                name="D_af",
                                info="Diffusivity of angiogenic factor",
                                real_value=D_af.magnitude,
                                real_um=D_af.units,
                                references="Estimated from Kihara, T., Ito, J., & Miyake, J. (2013). Measurement of Biomolecular Diffusion in Extracellular Matrix Condensed by Fibroblasts Using Fluorescence Correlation Spectroscopy. PLoS ONE, 8(11), 82382. https://doi.org/10.1371/JOURNAL.PONE.0082382",
                                sim_value=D_af.to("sau^2 / tau").magnitude,
                                sim_um=D_af.to("sau^2 / tau").units)

## D_af 
value:  4.2415938159916675e-05 millimeter ** 2 / second 



### Angiogenic factor uptake in capillaries ($V_{uC_{af}}$)

According to Vempati et al. (2014), the degradation rate of VEGF in cellular conditions is:

In [7]:
## Values reported by Vempati et al. (2014)
V_d_af_max = 2.8e-4 * ureg("1 / s")
V_d_af_max = V_d_af_max.to("1 / hr")
V_d_af_min = 2.3e-4 * ureg("1 / s")
V_d_af_min = V_d_af_min.to("1 / hr")

# Take the average
V_d_af = (V_d_af_max + V_d_af_min) / 2

print(f"## V_d_af \n"
      f"value:  {V_d_af} \n"
      f"range:  [{V_d_af_min}, {V_d_af_max}]")

## V_d_af 
value:  0.918 / hour 
range:  [0.8280000000000001 / hour, 1.008 / hour]


It is safe to assume that $V_{uC_{af}}$ is higher than the natural degradation rate of the angiogenic factors. So, we can state:

In [8]:
V_uC_af_min = V_d_af_min
V_uC_af = 1e2 * V_uC_af_min
V_uC_af_max = 1e4 * V_uC_af_min

# add it to parameters dict
parameters_df = fill_parameters(parameters_df,
                                name="V_uc_af",
                                info="Angiogenic factor uptake in capillaries",
                                real_value=V_uC_af.magnitude,
                                real_range_min=V_uC_af_min,
                                real_range_max=V_uC_af_max,
                                real_um=V_uC_af.units,
                                references="Estimated by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                sim_value=V_uC_af.to("1 / tau").magnitude,
                                sim_um="1 / tau",
                                sim_range_min=V_uC_af_min.to("1 / tau"),
                                sim_range_max=V_uC_af_max.to("1 / tau"))

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1784: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


## Initial number of source cells
We assume: 
- $af_min$, the value of the angiogenic factors outside the source cells.
- $af_max$, the value of the angiogenic factors inside the source cells.

In [9]:
n_source_cells = 300

parameters_df = fill_parameters(parameters_df,
                                name="n_source_cells",
                                info="Initial number of source cells",
                                real_value=n_source_cells,
                                real_um="[adim]",
                                references="",
                                sim_value=n_source_cells,
                                sim_um="[adim]")

## Source cells range
Following Travasso et al. (2011), we assume: 

In [10]:
source_cells_range = 0.025 * ureg.mm

parameters_df = fill_parameters(parameters_df,
                                name="source_cells_range",
                                info="Deactivation range for the hypoxic (source) cells",
                                real_value=source_cells_range.magnitude,
                                real_um=source_cells_range.units,
                                references="Estimated by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                sim_value=source_cells_range.to("sau").magnitude,
                                sim_um="sau")

## Max and Min value for the angiogenic factors (af)
We assume: 

In [11]:
af_min_real = 0 * ureg("ng / mL")
af_min_sim = af_min_real.to("afau")
af_max_real = 10 * ureg("ng / mL")
af_max_sim = af_max_real.to("afau")

parameters_df = fill_parameters(parameters_df,
                                name="af_min",
                                info=" Min value for angiogenic factor (outside hypoxic cells)",
                                real_value=af_min_real.magnitude,
                                real_um=af_min_real.units,
                                references="Estimated by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                sim_value=af_min_sim.magnitude,
                                sim_um=af_min_sim.units)

parameters_df = fill_parameters(parameters_df,
                                name="af_max",
                                info=" Max value for angiogenic factor (inside hypoxic cells)",
                                real_value=af_max_real.magnitude,
                                real_um=af_max_real.units,
                                references="Estimated by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                sim_value=af_max_sim.magnitude,
                                sim_um=af_max_sim.units)

## Angiogenesis parameters

### Cell radius ($R_c$)

For the cell radius we assumed half of the maximum value provided by Bionumbers (Milo et al., 2010) at the ID 100432, which is:

In [12]:
R_c = 10 * ureg.um

print(f"R_c: {R_c}")

# add it to parameters dict
parameters_df = fill_parameters(parameters_df,
                                name="R_c",
                                info="Cell radius",
                                real_value=R_c.magnitude,
                                real_um=R_c.units,
                                references="Milo et al. Nucl. Acids Res. (2010) 38 (suppl 1): D750-D753.; ID: 100432",
                                sim_value=R_c.to("sau").magnitude,
                                sim_um=R_c.to("sau").units)

R_c: 10 micrometer


This value also agrees with the endothelial cells dimension which can be inferred from the pictures reported by Shamloo, A. et al. (2008) (see Fig. 5A and 5B).

### Interface width of the capillaries ($\epsilon$)

The capillaries phase field model depends on a parameters $\epsilon$, regulating the interfacial width between the capillary and the surrounding tissue. We use the same value used by Travasso et al. (2011):

In [13]:
epsilon = (1.25 ** 2) * (ureg.um ** 2)  # notice: in Travasso provided as 'um' but it is probably an error

epsilon_real = epsilon
epsilon_sim = epsilon.to('sau ** 2')
print(f"epsilon: {epsilon_real:.2g}\n"
      f"         {epsilon_sim:.2g}")

epsilon: 1.6 micrometer ** 2
         1.6e-06 SpaceArbitraryUnit ** 2


In [14]:
# add it to parameters
parameters_df = fill_parameters(parameters_df,
                                name="epsilon",
                                info="Interface width",
                                real_value=epsilon_real.magnitude,
                                real_um=epsilon_real.units,
                                references="Same provied by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                sim_value=epsilon_sim.magnitude,
                                sim_um=epsilon_sim.units)

### Time step ($dt$)
The simulation time step is a parameter of the angiogenesis model because it regulates the frequency of tip cell activation. Indeed, at each time step maximum one tip cell can activate. We assume $dt$ to be the same as the arbitrary time unit:

In [15]:
dt = sim_t_au
print(f"dt: {dt}")

dt: 26 minute


This makes sure that the activation of new tip cells does not lead to discontinuous capillaries, because the maximum cell displacement (i.e. the maximum tip cells' velocity times $dt$) is lower than a TC radius (see section regarding $G_M$)

In [16]:
parameters_df = fill_parameters(parameters_df,
                                name="dt",
                                info="Simulation time step",
                                real_value=dt.magnitude,
                                real_um=dt.units,
                                references="Same provied by Travasso, R. D. M., Poire, E. C., Castro, M., Rodrguez-Manzaneque, J. C., & Hernandez-Machado, A. (2011). Tumor angiogenesis and vascular patterning: A mathematical model. PLoS ONE, 6(5), e19989. https://doi.org/10.1371/journal.pone.0019989",
                                sim_value=dt.to("tau").magnitude,
                                sim_um=dt.to("tau").units)

### Angiogenic factor value for maximum proliferation ($T_p$)
Song, M. & Finley, S. D. (2022), report that endothelial cells reach their maximum proliferation potential for a VEGF concentration of $1 [\frac{ng}{mL}]$.

In [17]:
T_p_real = 1 * ureg("ng / mL")

This value allowed them to obtain results comparable with angiogenic experiments. Thus, we kept the same.

In [18]:
T_p_sim = T_p_real.to("afau")

parameters_df = fill_parameters(parameters_df,
                                name="T_p",
                                info="Angiogenic factor concentration for maximum proliferation",
                                real_value=T_p_real.magnitude,
                                real_um=T_p_real.units,
                                references="None",
                                sim_value=T_p_sim.magnitude,
                                sim_um=T_p_sim.units)

### Endothelial cells proliferation rate ($\alpha_p$)

Based on the measures by Tracqui, P. (2005), Travasso et al. (2011) estimated a normal value for the maximum proliferation rate of endothelial cells to be:


In [19]:
endothelial_cells_proliferation_rate = 0.97 * (1 / ureg.hr)

Pairing this value to the *in silico* maximum proliferation rate, which is $\alpha_p T_p$, we get:

In [20]:
alpha_p = endothelial_cells_proliferation_rate.to("1 / tau") / T_p_sim
alpha_p = alpha_p.to("1 / (tau * afau)")
print(f"alpha_p: {alpha_p:.2g}")

alpha_p: 0.42 / AFsArbitraryUnit / TimeArbitraryUnit


In [21]:
alpha_p_real = alpha_p.to("(mL / pg) * (1 / hr)")
alpha_p_sim = alpha_p

In [22]:
parameters_df = fill_parameters(parameters_df,
                                name="alpha_p",
                                info="Stalk cells proliferation rate",
                                real_value=alpha_p_real.magnitude,
                                real_um=alpha_p_real.units,
                                sim_value=alpha_p_sim.magnitude,
                                sim_um=alpha_p_sim.units)

Here we also separate the proliferation rate of the stalk cells from the proliferation rate of the endothelial cells of the capillaries:

In [23]:
parameters_df = fill_parameters(parameters_df,
                                name="alpha_pc",
                                info="Endothelial cells proliferation rate",
                                real_value=alpha_p_real.magnitude,
                                real_um=alpha_p_real.units,
                                sim_value=alpha_p_sim.magnitude,
                                sim_um=alpha_p_sim.units)

### Minimum angiogenic factor concentration for tip cells activation ($T_c$)

Despite difficult to measure, a threshold in AF concentration for tip cell activation is likely to exist, since they need to bind to cellular surface receptors to influence cells behaviour. Song M. & Finley S.D. (2022) observed a shift in VEGF-mediated sprouting angiogenesis in correspondence of a VEGF concentration of $3 [\frac{ng}{mL}]$:

In [24]:
T_c_real = 3 * ureg("ng / mL")
T_c_sim = T_c_real.to("afau")

print(f"T_c: {T_c_real} (real) | {T_c_sim} (sim)")

parameters_df = fill_parameters(parameters_df,
                                name="T_c",
                                info="Minimum af for Tip Cells Activation",
                                real_value=T_c_real.magnitude,
                                real_um=T_c_real.units,
                                sim_value=T_c_sim.magnitude,
                                sim_um=T_c_sim.units,
                                references="Song, M. & Finley, S.D.Mechanistic characterization of endothelial sprouting mediated by pro-angiogenic signaling. Microcirculation 29, e12744(2022).")

T_c: 3.0 nanogram / milliliter (real) | 3.000000000000001 AFsArbitraryUnit (sim)


### Mimimum angiogenic factor gradient for Tip Cells activation ($G_m$)
From Shamloo et al (2008) we have:

In [25]:
G_m_real = 14 * ureg("ng / (mL * mm)")
# converting to arbitrary units:
G_m_sim = G_m_real.to("afau / sau")
print(f"## G_m\n"
      f"real: {G_m_real}\n"
      f"sim : {G_m_sim}")

## G_m
real: 14.0 nanogram / milliliter / millimeter
sim : 14.000000000000004 AFsArbitraryUnit / SpaceArbitraryUnit


In [26]:
parameters_df = fill_parameters(parameters_df,
                                name="G_m",
                                info="Minimum af gradient required for Tip Cells Activation",
                                real_value=G_m_real.magnitude,
                                real_um=G_m_real.units,
                                sim_value=G_m_sim.magnitude,
                                sim_um=G_m_sim.units,
                                references="Shamloo, A., Ma, N., Poo, M. M., Sohn, L. L., & Heilshorn, S. C. (2008). Endothelial cell polarization and chemotaxis in a microfluidic device. Lab on a Chip, 8(8), 1292-1299. https://doi.org/10.1039/B719788H")

### Minimum value for capillaries field c to allow Tip Cells activation ($\varphi_{th}$)

Tip cells can originate only inside the already formed capillaries. Thus, we set:

In [27]:
phi_th = 0.9

And have:

In [28]:
parameters_df = fill_parameters(parameters_df,
                                name="phi_th",
                                info="Minimum capillaries field value for Tip Cells activation",
                                sim_value=phi_th)

### Gradient for maximum tip cells velocity ($G_M$)
According to Travasso et al. (2011):

In [29]:
G_M_real = 3 * G_m_real
G_M_sim = 3 * G_m_sim

print(f"G_M: \n"
      f"     (real) {G_M_real:.2g}\n"
      f"     (sim)  {G_M_sim:.2g}")

G_M: 
     (real) 42 nanogram / milliliter / millimeter
     (sim)  42 AFsArbitraryUnit / SpaceArbitraryUnit


This value is estimated by Stokes et al. (1991):

In [30]:
parameters_df = fill_parameters(parameters_df,
                                name="G_M",
                                info="af gradient for maximum tip cells velocity",
                                real_value=G_M_real.magnitude,
                                real_um=G_M_real.units,
                                sim_value=G_M_sim.magnitude,
                                sim_um=G_M_sim.units)

### Extreme values for capillaries field ($\varphi_{min}$ and $\varphi_{max}$)

For the system construction, these values are just 1 and -1:

In [31]:
phi_min = -1  # define minimum phi value

# add it to parameters
parameters_df = fill_parameters(parameters_df,
                                name="phi_min",
                                info="Min capillaries field value",
                                sim_value=phi_min)

phi_max = 1  # define maximum phi value

# add it to parameters
parameters_df = fill_parameters(parameters_df,
                                name="phi_max",
                                info="Max capillaries field value",
                                sim_value=phi_max)

### Motility value for capillaries phase field ($M$)

Travasso et al. assume this value to be $10^{-15} [\frac{m^2}{s}]$. We assume the same:

In [32]:
M_real = 1e-15 * ureg("m ** 2 / s")
M_real = M_real.to("mm ** 2 / s")
M_sim = M_real.to("sau ** 2 / tau")

parameters_df = fill_parameters(parameters_df,
                                name="M",
                                info="'Motility' for the capillaries phase field",
                                real_value=M_real.magnitude,
                                real_um=M_real.units,
                                sim_value=M_sim.magnitude,
                                sim_um=M_sim.units)

### Chemotactic coefficient for tip cells ($\chi$)

Travasso assumes that the maximum tip cells velocity is in the order of (Figure 1; Travasso et al. 2011):

In [33]:
max_tip_cells_velocity_real = 0.35 * (ureg.um / ureg.min)
print(f"max tip cells velocity (real): {max_tip_cells_velocity_real:.2g}")
max_tip_cells_velocity_sim = max_tip_cells_velocity_real.to("sau / tau")
print(f"max tip cells velocity (sim): {max_tip_cells_velocity_sim:.2g}")

max tip cells velocity (real): 0.35 micrometer / minute
max tip cells velocity (sim): 0.0091 SpaceArbitraryUnit / TimeArbitraryUnit


And recent experiments confirm this result. Blinder Y.J. et al. (2015) report an average tip cell velocity of $17 \frac{\mu m}{hr} =  0.28 \frac{\mu m}{min}$.

This leads to a $\chi$ value of:

In [34]:
chi_sim = max_tip_cells_velocity_sim / G_M_sim
print(f"chi (sim): {chi_sim:.2g}")
chi_real = max_tip_cells_velocity_real / G_M_real
print(f"chi (real): {chi_real:.2g}")

chi (sim): 0.00022 SpaceArbitraryUnit ** 2 / AFsArbitraryUnit / TimeArbitraryUnit
chi (real): 0.0083 micrometer * milliliter * millimeter / minute / nanogram


In [35]:
parameters_df = fill_parameters(parameters_df,
                                name="chi",
                                info="chemotactic constant for tip cells",
                                real_value=chi_real.magnitude,
                                real_um=chi_real.units,
                                sim_value=chi_sim.magnitude,
                                sim_um=chi_sim.units)

### Minimum Tip Cells distance
The generation of new Tip Cells is inhibited by other Tip Cells due to the Notch Pathway. This mechanism prevents the formation of two neighbor tip cells, thus, Travasso et al. (2011) assumed that exists a minimum distance for the activation of two tip cells equal to 4 cell radiuses:

In [36]:
min_tipcell_distance_real = 4 * R_c
min_tipcell_distance_sim = min_tipcell_distance_real.to("sau")

print(f"min tipcell distance: \n"
      f"                      (real) {min_tipcell_distance_real:.2g}\n"
      f"                      (sim)  {min_tipcell_distance_sim:.2g}")

min tipcell distance: 
                      (real) 40 micrometer
                      (sim)  0.04 SpaceArbitraryUnit


In [37]:
parameters_df = fill_parameters(parameters_df,
                                name="min_tipcell_distance",
                                info="Min Tip Cells distance (due to Notch pathway)",
                                real_value=min_tipcell_distance_real.magnitude,
                                real_um=min_tipcell_distance_real.units,
                                sim_value=min_tipcell_distance_sim.magnitude,
                                sim_um=min_tipcell_distance_sim.units)

## Tip Cell Frequency


In [38]:
tc_freq_sim = 1 * ureg("1 / tau")
tc_freq_real = tc_freq_sim.to("1 / hours")


parameters_df = fill_parameters(parameters_df,
                                name="tc_freq",
                                info="Frequency of TC activation",
                                real_value=tc_freq_real.magnitude,
                                real_um=tc_freq_real.units,
                                sim_value=tc_freq_sim.magnitude,
                                sim_um=tc_freq_sim.units)

# Complete parameters
The complete parameters table is:

In [39]:
from IPython.display import display

parameters_df.set_index("name", inplace=True)  # needed for simulation

display(parameters_df)

,info,real_value,real_range_max,real_range_min,real_um,sim_value,sim_range_max,sim_range_min,sim_um,references,influence
name,,,,,,,,,,,
SpaceArbitraryUnit,None,1,None,None,millimeter,1.000000,None,None,SpaceArbitraryUnit,None,None
TimeArbitraryUnit,None,26,None,None,minute,1.000000,None,None,TimeArbitraryUnit,None,None
AFsArbitraryUnit,None,1.0,None,None,nanogram / milliliter,1.000000,None,None,afau,None,None
D_af,Diffusivity of angiogenic factor,0.000042,None,None,millimeter ** 2 / second,0.066169,None,None,SpaceArbitraryUnit ** 2 / TimeArbitraryUnit,"Estimated from Kihara, T., Ito, J., & Miyake, ...",None
V_uc_af,Angiogenic factor uptake in capillaries,82.8,8280.0 / hour,0.8280000000000001 / hour,1 / hour,35.880000,3587.9999999999995 / TimeArbitraryUnit,0.3588 / TimeArbitraryUnit,1 / tau,"Estimated by Travasso, R. D. M., Poire, E. C.,...",None
n_source_cells,Initial number of source cells,300,None,None,[adim],300.000000,None,None,[adim],,None
source_cells_range,Deactivation range for the hypoxic (source) cells,0.025,None,None,millimeter,0.025000,None,None,sau,"Estimated by Travasso, R. D. M., Poire, E. C.,...",None
af_min,Min value for angiogenic factor (outside hypo...,0.0,None,None,nanogram / milliliter,0.000000,None,None,AFsArbitraryUnit,"Estimated by Travasso, R. D. M., Poire, E. C.,...",None
af_max,Max value for angiogenic factor (inside hypox...,10.0,None,None,nanogram / milliliter,10.000000,None,None,AFsArbitraryUnit,"Estimated by Travasso, R. D. M., Poire, E. C.,...",None


So we can convert it to csv:

In [40]:
parameters_df.to_csv("parameters.csv")

# References

Lai, X., & Friedman, A. (2019). Mathematical modeling in scheduling cancer treatment with combination of VEGF inhibitor and chemotherapy drugs. Journal of Theoretical Biology, 462, 490-498. https://doi.org/10.1016/J.JTBI.2018.11.018

Liao, K.L. , Bai, X.F. , Friedman, A. , 2014. Mathematical modeling of interleukin-27 induction of anti-tumor t cells response. PLoS One 9 (3), e91844.

Guerra, A., Belinha, J., Mangir, N., MacNeil, S., & Natal Jorge, R. (2021). Sprouting Angiogenesis: A Numerical Approach with Experimental Validation. Annals of Biomedical Engineering, 49(2), 871-884. https://doi.org/10.1007/S10439-020-02622-W/TABLES/1

Phillips, C. M., Lima, E. A. B. F., Woodall, R. T., Brock, A., & Yankeelov, T. E. (2020). A hybrid model of tumor growth and angiogenesis: In silico experiments. PLOS ONE, 15(4), e0231137. https://doi.org/10.1371/JOURNAL.PONE.0231137

Levine, H. A., Pamuk, S., Bölümü, M., Sleeman, B. D., Nilsen-Hamilton, M., Cotran, R. A. S., Leapman, S. B., Folkman, J., & Levine, H. A. (2001). Mathematical modeling of capillary formation and development in tumor angiogenesis: Penetration into the stroma. Bulletin of Mathematical Biology 2001 63:5, 63(5), 801-863. https://doi.org/10.1006/BULM.2001.0240

Kihara, T., Ito, J., & Miyake, J. (2013). Measurement of Biomolecular Diffusion in Extracellular Matrix Condensed by Fibroblasts Using Fluorescence Correlation Spectroscopy. PLoS ONE, 8(11), 82382. https://doi.org/10.1371/JOURNAL.PONE.0082382

Kut, C., Mac Gabhann, F., & Popel, A. S. (2007). Where is VEGF in the body? A meta-analysis of VEGF distribution in cancer. British Journal of Cancer 2007 97:7, 97(7), 978-985. https://doi.org/10.1038/sj.bjc.6603923

Milo et al. Nucl. Acids Res. (2010) 38 (suppl 1): D750-D753.

Tracqui, P., Liu, J. W., Collin, O., Clement-Lacroix, J., & Planus, E. (2005). Global analysis of endothelial cell line proliferation patterns based on nutrient-depletion models: implications for a standardization of cell proliferation assays. Cell Proliferation, 38(3), 119-135. https://doi.org/10.1111/J.1365-2184.2005.00337.X

Los, M., Aarsman, C. J. M., Terpstra, L., Wittebol-Post, D., Lips, C. J. M., Blijham, G. H., & Voest, E. E. (1997). Elevated ocular levels of vascular endothelial growth factor in patients with von Hippel-Lindau disease. Annals of Oncology, 8(10), 1015-1022. https://doi.org/10.1023/A:1008213320642

Karayiannakis, A. J., Zbar, A., Polychronidis, A., & Simopoulos, C. (2003). Serum and Drainage Fluid Vascular Endothelial Growth Factor Levels in Early Surgical Wounds. European Surgical Research, 35(6), 492-496. https://doi.org/10.1159/000073388

Shamloo, A., Ma, N., Poo, M. M., Sohn, L. L., & Heilshorn, S. C. (2008). Endothelial cell polarization and chemotaxis in a microfluidic device. Lab on a Chip, 8(8), 1292-1299. https://doi.org/10.1039/B719788H

Stokes, C. L., Lauffenburger, D. A., & Williams, S. K. (1991). Migration of individual microvessel endothelial cells: stochastic model and parameter measurement.

Finley, S. D., Dhar, M., & Popel, A. S. (2013). Compartment model predicts VEGF secretion and investigates the effects of VEGF Trap in tumor-bearing mice. Frontiers in Oncology, 3 JUL, 196. https://doi.org/10.3389/fonc.2013.00196

Na, X., Wu, G., Ryan, C. K., Schoen, S. R., Di’Santagnese, P. A., & Messing, E. M. (2003). Overproduction of vascular endothelial growth factor related to von Hippel-Lindau tumor suppressor gene mutations and hypoxia-inducible factor-1α expression in renal cell carcinomas. Journal of Urology, 170(2 I), 588-592. https://doi.org/10.1097/01.ju.0000074870.54671.98

Blinder, Y. J., Freiman, A., Raindel, N., Mooney, D. J., & Levenberg, S. (2015). Vasculogenic dynamics in 3D engineered tissue constructs. Scientific Reports, 5(1), 1-8. https://doi.org/10.1038/srep17840

Goswami, A., Surve, A., & Venkatesh, P. (2021). Optical Coherence Tomography Angiography of Early Stage 1a Retinal Hemangioblastoma in Von-Hippel-Lindau. Journal of Kidney Cancer and VHL, 8(3), 15-18. https://doi.org/10.15586/JKCVHL.V8I3.158

Arora, S., Chung, H., & Damji, K. (2015). Ophthalmology and Clinical Research Optic Disk Size Assessment Techniques: Photo Essay. Int J Ophthalmol Clin Res, 2, 1.

Ye, D. Y., Bakhtian, K. D., Asthagiri, A. R., & Lonser, R. R. (2012). Effect of pregnancy on hemangioblastoma development and progression in von Hippel-Lindau disease: Clinical article. Journal of Neurosurgery, 117(5), 818-824. https://doi.org/10.3171/2012.7.JNS12367

Imoukhuede, P. I., & Popel, A. S. (2011). Quantification and cell-to-cell variation of vascular endothelial growth factor receptors. Experimental Cell Research, 317(7), 955-965. https://doi.org/10.1016/J.YEXCR.2010.12.014

Bateman, A., Martin, M. J., Orchard, S., Magrane, M., Agivetova, R., Ahmad, S., Alpi, E., Bowler-Barnett, E. H., Britto, R., Bursteinas, B., Bye-A-Jee, H., Coetzee, R., Cukura, A., da Silva, A., Denny, P., Dogan, T., Ebenezer, T. G., Fan, J., Castro, L. G., … Teodoro, D. (2021). UniProt: the universal protein knowledgebase in 2021. Nucleic Acids Research, 49(D1), D480-D489. https://doi.org/10.1093/NAR/GKAA1100

Phillips, C. M. et al. Towards integration of time-resolved confocal microscopy of a 3D in vitro microfluidic platform with a hybrid multiscale model of tumor angiogenesis. PLOS Comput. Biol. 19, e1009499 (2023).

MACDOUGALL, J., MCCABE, M. Diffusion Coefficient of Oxygen through Tissues. Nature 215, 1173–1174 (1967). https://doi.org/10.1038/2151173a0

Song, M. & Finley, S. D. Mechanistic characterization of endothelial sprouting mediated by pro-angiogenic signaling. Microcirculation 29, e12744 (2022).